This notebook describes a synthetic optimisation experiment. Specifically, we try to maximise the expectation
$$
\mathbb{E}_{\mathbf{X} \sim p(\mathbf{X})}\left[
    \frac{1}{D}\sum_{i = 1}^D (X_i - 0.499)^2
\right],
$$
using different gradient estimators. We choose to set $D = 200$. While being a synthetic example, this optimisation task is already challenging because of the small impact of on the overal expectation value of changing a single variable.
We optimised the hyperparameters for each method, which are the following for each method.

1. `LR = 5.` for IndeCateR ('icr') with `SAMPLES = 2`
2. `LR = 5.` for RLOO ('rloo')' with `SAMPLES = 800` (RLOO-F)
3. `LR = 1.` for RLOO ('rloo')' with `SAMPLES = 2` (RLOO-S)
4. `LR = 0.01` for Gumbel-Softmax ('gs) with `SAMPLES = 800`, `TEMP = 0.1` and `ANNEAL_RATE = 0.05`. We only compare Gumbel-Softmax with 800 samples, because it was already very unstable because of the challening problem.

In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt

from losses import sq_loss
from jointcategorical import JointCategorical


os.environ["CUDA_VISIBLE_DEVICES"] = "3"

DIM = 200
CATS = 2
SAMPLES = 2
TEMP = 0.1
ANNEAL_RATE = 0.05
LR = 5.
OPTIM = 'rms'

### RLOO-800 and ICR LR 5
### RLOO-2 LR 1.0
### GS LR 0.01 and TEMP 0.1

The next cell performs 2000 iterations of optimisations using the desired gradient type and stores the training results for 10 independent runs.

In [3]:
for grad in ['icr']:
    for seed in range(10):
        joint = JointCategorical(DIM, CATS, SAMPLES, anneal_rate=ANNEAL_RATE, temp=TEMP, grad_type=grad)

        print("Initial loss:", joint.grads(sq_loss, None)[0])
        joint.optimiser = tf.keras.optimizers.RMSprop(LR)
        joint.train(2000, loss=sq_loss, target=None, log_its=5)

        if grad == 'gs':
            pickle.dump(joint.logger, open(f"results_s2/{grad}_temp{TEMP}_a{ANNEAL_RATE}_s{SAMPLES}_d{DIM}_o{OPTIM}_lr{LR}_{seed}_nonorm.pkl", "wb"))    
        pickle.dump(joint.logger, open(f"results_s2/{grad}_s{SAMPLES}_d{DIM}_o{OPTIM}_lr{LR}_{seed}_nonorm.pkl", "wb"))

2023-12-11 15:08:53.915542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-11 15:08:53.915575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ZBook-LennertDTAI
2023-12-11 15:08:53.915579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ZBook-LennertDTAI
2023-12-11 15:08:53.915702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.147.5
2023-12-11 15:08:53.915719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.147.5
2023-12-11 15:08:53.915722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.147.5
2023-12-11 15:08:53.916116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is o

Initial loss: tf.Tensor(-0.25000602, shape=(), dtype=float32)
Iterations 5: mean function -0.25013601779937744 Time (s): 0.018177509307861328 Gradient variance: 3.153829122049904e-12
Iterations 10: mean function -0.2502550184726715 Time (s): 0.011396408081054688 Gradient variance: 3.4891495075128898e-12
Iterations 15: mean function -0.2504020035266876 Time (s): 0.012005329132080078 Gradient variance: 3.0775961206569447e-12
Iterations 20: mean function -0.25049400329589844 Time (s): 0.013675689697265625 Gradient variance: 2.6490784392485534e-12
Iterations 25: mean function -0.250588983297348 Time (s): 0.012252092361450195 Gradient variance: 2.2703594646650282e-12
Iterations 30: mean function -0.25062698125839233 Time (s): 0.012575864791870117 Gradient variance: 1.953133835219667e-12
Iterations 35: mean function -0.2506759762763977 Time (s): 0.011400222778320312 Gradient variance: 1.6881945060637227e-12
Iterations 40: mean function -0.2507140040397644 Time (s): 0.01153874397277832 Gradie

KeyboardInterrupt: 

In [ ]:
red_salsa = '#F94144'  # main red colour
orange_red = '#F3722C'
yellow_orange= '#F8961E'
mango_tango = '#F9844A'
maize_crayola = '#F9C74F'  # yellowish
pistachio = '#90BE6D'  # greenish colour
jungle_green = '#43AA8B'
steel_teal = '#4D908E'
queen_blue = '#577590'
celadon_blue = '#277DA1'  # main blue colour
pink = '#FA39FA'

Once all configurations are optimised, the next couple of cells load the stored optimisation results. These results, being expected function value and gradient variance throughout training, can then be plotted in function of time and iterations.

In [ ]:
GRAD = 'rloo'
SAMPLES = 800

train_losses = []
variances = []
time = []
for i in range(10):
    logger = pickle.load(open(f"results_s2/{GRAD}_s{SAMPLES}_d{DIM}_o{OPTIM}_lr{5.}_{i}_nonorm.pkl", "rb"))
    train_losses.append(list(logger.log_dict["training_loss"].values()))
    variances.append(list(logger.log_dict["gradient_variance"].values()))
    time.append(list(logger.log_dict["time"].values()))

In [ ]:
GRAD = 'icr'

train_losses2 = []
variances2 = []
time2 = []
for i in range(10):
    logger = pickle.load(open(f"results_s2/{GRAD}_s{2}_d{DIM}_o{OPTIM}_lr{5.}_{i}_nonorm.pkl", "rb"))
    train_losses2.append(list(logger.log_dict["training_loss"].values()))
    variances2.append(list(logger.log_dict["gradient_variance"].values()))
    time2.append(list(logger.log_dict["time"].values()))

In [ ]:
GRAD = 'rloo'

train_losses4 = []
variances4 = []
time4 = []
for i in range(10):
    logger = pickle.load(open(f"results_s2/{GRAD}_s{2}_d{DIM}_o{OPTIM}_lr{1.}_{i}_nonorm.pkl", "rb"))
    train_losses4.append(list(logger.log_dict["training_loss"].values()))
    variances4.append(list(logger.log_dict["gradient_variance"].values()))
    time4.append(list(logger.log_dict["time"].values()))

In [ ]:
GRAD = 'gs'
TEMP = 0.1

train_losses3 = []
variances3 = []
time3 = []
for i in range(10):
    logger = pickle.load(open(f"results_s2/{GRAD}_temp{TEMP}_a{ANNEAL_RATE}_s800_d200_orms_lr{1e-2}_{i}_nonorm.pkl", "rb"))
    train_losses3.append(list(logger.log_dict["training_loss"].values()))
    variances3.append(list(logger.log_dict["gradient_variance"].values()))
    time3.append(list(logger.log_dict["time"].values()))

In [ ]:
train_avg = -tf.reduce_mean(train_losses, axis=0)
train_std = tf.math.reduce_std(train_losses, axis=0) / np.sqrt(10)
train_avg2 = -tf.reduce_mean(train_losses2, axis=0)
train_std2 = tf.math.reduce_std(train_losses2, axis=0) / np.sqrt(10)
train_avg3 = -tf.reduce_mean(train_losses3, axis=0)
train_std3 = tf.math.reduce_std(train_losses3, axis=0) / np.sqrt(10)
train_avg4 = -tf.reduce_mean(train_losses4, axis=0)
train_std4 = tf.math.reduce_std(train_losses4, axis=0) / np.sqrt(10)
its = [i * 5 for i in range(len(train_avg))]

In [ ]:
train_avg = -tf.reduce_mean(train_losses, axis=0)
train_std = tf.math.reduce_std(train_losses, axis=0) / np.sqrt(10)
train_avg2 = -tf.reduce_mean(train_losses2, axis=0)
train_std2 = tf.math.reduce_std(train_losses2, axis=0) / np.sqrt(10)
train_avg3 = -tf.reduce_mean(train_losses3, axis=0)
train_std3 = tf.math.reduce_std(train_losses3, axis=0) / np.sqrt(10)
train_avg4 = -tf.reduce_mean(train_losses4, axis=0)
train_std4 = tf.math.reduce_std(train_losses4, axis=0) / np.sqrt(10)

its = tf.range(len(train_avg), dtype=tf.float32)
its1 = its * np.mean(np.mean(time, axis=0), axis=0)
its2 = its * np.mean(np.mean(time2, axis=0), axis=0)
its3 = its * np.mean(np.mean(time3, axis=0), axis=0)
its4 = its * np.mean(np.mean(time4, axis=0), axis=0)

min_time = tf.reduce_min([its1[-1], its2[-1], its3[-1], its4[-1]])

In [ ]:
var_avg = tf.reduce_mean(variances, axis=0)
var_std = tf.math.reduce_std(variances, axis=0) / np.sqrt(10)
var_avg2 = tf.reduce_mean(variances2, axis=0)
var_std2 = tf.math.reduce_std(variances2, axis=0) / np.sqrt(10)
var_avg3 = tf.reduce_mean(variances3, axis=0)
var_std3 = tf.math.reduce_std(variances3, axis=0) / np.sqrt(10)
var_avg4 = tf.reduce_mean(variances4, axis=0)
var_std4 = tf.math.reduce_std(variances4, axis=0) / np.sqrt(10)

In [ ]:
its = [i * 5 for i in range(len(train_avg))]

fig = plt.figure(figsize=(24, 6))
ax1 = fig.add_subplot(131)
ax1.set_xlabel('Iterations', fontsize=16)
ax1.set_ylabel('Function Value', fontsize=16)
ax1.plot(its, train_avg2, color=jungle_green, label='IndeCateR')
ax1.fill_between(its, train_avg2 - train_std2, train_avg2 + train_std2, alpha=0.33, color=jungle_green)
ax1.plot(its, train_avg, color=pink, label='RLOO-800', linestyle='dashed')
ax1.fill_between(its, train_avg - train_std, train_avg + train_std, alpha=0.33, color=pink, linestyle='dashed')
ax1.plot(its, train_avg4, color=celadon_blue, label='RLOO-2', linestyle='dashed')
ax1.fill_between(its, train_avg4 - train_std4, train_avg4 + train_std4, alpha=0.33, color=celadon_blue, linestyle='dashed')
ax1.plot(its, train_avg3, color=orange_red, label='GS', linestyle='dotted')
ax1.fill_between(its, train_avg3 - train_std3, train_avg3 + train_std3, alpha=0.33, color=orange_red, linestyle='dotted')
ax1.spines[['right', 'top']].set_visible(False)

ax2 = fig.add_subplot(132)
ax2.set_xlabel('Time (s)', fontsize=16)
ax2.set_ylabel('Function Value', fontsize=16)
ax2.plot(its2, train_avg2, color=jungle_green, label='IndeCateR')
ax2.fill_between(its2, train_avg2 - train_std2, train_avg2 + train_std2, alpha=0.33, color=jungle_green)
ax2.plot(its1, train_avg, color=pink, label='RLOO-F', linestyle='dashed')
ax2.fill_between(its1, train_avg - train_std, train_avg + train_std, alpha=0.33, color=pink, linestyle='dashed')
ax2.plot(its4, train_avg4, color=celadon_blue, label='RLOO-S', linestyle='dashed')
ax2.fill_between(its4, train_avg4 - train_std4, train_avg4 + train_std4, alpha=0.33, color=celadon_blue, linestyle='dashed')
ax2.plot(its3, train_avg3, color=orange_red, label='GS-F', linestyle='dotted')
ax2.fill_between(its3, train_avg3 - train_std3, train_avg3 + train_std3, alpha=0.33, color=orange_red, linestyle='dotted')
ax2.spines[['right', 'top']].set_visible(False)
ax2.set_xlim(xmax=min_time)

ax3 = fig.add_subplot(133)
ax3.set_xlabel('Iterations', fontsize=16)
ax3.set_ylabel('Gradient Variance', fontsize=16)
ax3.plot(its, var_avg2, color=jungle_green, label='IndeCateR')
ax3.fill_between(its, var_avg2 - var_std2, var_avg2 + var_std2, alpha=0.33, color=jungle_green)
ax3.plot(its, var_avg, color=pink, label='RLOO-F', linestyle='dashed')
ax3.fill_between(its, var_avg - var_std, var_avg + var_std, alpha=0.33, color=pink, linestyle='dashed')
ax3.plot(its, var_avg4, color=celadon_blue, label='RLOO-S', linestyle='dashed')
ax3.fill_between(its, var_avg4 - var_std4, var_avg4 + var_std4, alpha=0.33, color=celadon_blue, linestyle='dashed')
ax3.plot(its, var_avg3, color=orange_red, label='GS-F', linestyle='dotted')
ax3.fill_between(its, var_avg3 - var_std3, var_avg3 + var_std3, alpha=0.33, color=orange_red, linestyle='dotted')
ax3.spines[['right', 'top']].set_visible(False)
ax3.set_yscale('log')

leg = plt.legend(frameon=False, fontsize=24, loc='upper center', bbox_to_anchor=(-.8, 1.2), ncol=4)
for line in leg.get_lines():
    line.set_linewidth(4)
plt.plot()
plt.savefig(f"/cw/dtaijupiter/NoCsBack/dtai/lennert/CATSCH/synthetic/plots/synthetic_optimisation.pdf", dpi=300, bbox_inches='tight', transparent=True)
plt.show()